In [1]:
# 1、加载模型和预训练权重
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = r"F:\pythonProject\google_bert\bert_base_uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at F:\pythonProject\google_bert\bert_base_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

看到这么一大串的warning出现，不要怕，这个warning正是我们希望看到的。

为啥会出现这个warning呢，因为我们加载的预训练权重是bert-based-uncased，而使用的骨架是AutoModelForSequenceClassification，前者是没有在下游任务上微调过的，所以用带有下游任务Head的骨架去加载，会随机初始化这个Head。这些在warning中也说的很明白。

In [3]:
# 2、加载数据集
from datasets import load_from_disk

dataset_path = r"F:\pythonProject\datasets\glue\mrpc"
raw_datasets = load_from_disk(dataset_path)

In [4]:
# 3、查看整个数据集结构
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

load_dataset出来的是一个DatasetDict对象，它包含了train，validation，test三个属性。可以通过key来直接查询，得到对应的train、valid和test数据集。

这里的train，valid，test都是Dataset类型，有 features和num_rows两个属性。还可以直接通过下标来查询对应的本。

In [5]:
# 3.1查看单个样本数据结构
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [6]:
# 3.2查看单个样本的特征
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

Dataset的features可以理解为一张表的columns，Dataset甚至可以看做一个pandas的dataframe，二者的使用很类似。

In [7]:
# 4、数据集预处理
def tokenize_function(sample):
    # 这里可以添加多种操作，不光是tokenize
    # 这个函数处理的对象，就是Dataset这种数据类型，通过features中的字段来选择要处理的数据
    return tokenizer(sample['sentence1'], sample['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

官方推荐的做法是通过Dataset.map方法，来调用一个分词方法，实现批量化的分词。注意到，在这个tokenize_function中，没有使用padding，因为如果使用了padding之后，就会全局统一做一个maxlen进行padding，这样无论在tokenize还是模型的训练上都不够高效。实际上，我们是故意先不进行padding的，因为我们想在划分batch的时候再进行padding，这样可以避免出现很多有一堆padding的序列，从而可以显著节省我们的训练时间。这样就需要用到**DataCollatorWithPadding，来进行动态padding**

In [8]:
# 4.1 动态padding，collator数据
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

注意，我们需要使用tokenizer来初始化这个DataCollatorWithPadding，因为需要tokenizer来告知具体的padding token是啥，
以及padding的方式是在左边还是右边（不同的预训练模型，使用的padding token以及方式可能不同）。

In [9]:
# 5、数据导入Pytorch的Dataloader前，执行分词处理后样本数据的列表名称
print(tokenized_datasets['train'].column_names)

['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']


huggingface中datasets准备了三个方法：remove_columns, rename_column, set_format，方便为pytorch的Dataloader做准备:

* 去除‘sentence1’，‘sentence2’等
* 把数据转换成pytorch tensors
* 修改列名 label 为 labels

In [10]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2','idx'])
tokenized_datasets = tokenized_datasets.rename_column('label','labels')  
tokenized_datasets.set_format('torch')

In [11]:
print(tokenized_datasets['train'].column_names)

['labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [12]:
# 5.1定义pytorch dataloaders
from torch.utils.data import DataLoader, Dataset
# 通过这里的dataloader，每个batch的seq_len可能不同
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator)  
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator)

# 查看一下train_dataloader的元素长啥样
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 74]),
 'token_type_ids': torch.Size([8, 74]),
 'attention_mask': torch.Size([8, 74])}

按道理说，Huggingface这边提供Transformer模型就已经够了，具体的训练、优化，应该交给pytorch了吧。
但鉴于Transformer训练时，最常用的优化器就是AdamW，这里Huggingface也直接在transformers库中加入
了AdamW这个优化器，还贴心地配备了lr_scheduler，方便我们直接使用。

In [14]:
# 6、定义 optimizer 和 learning rate scheduler
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)  # num of batches * num of epochs
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,  # scheduler是针对optimizer的lr的
    num_warmup_steps=0,
    num_training_steps=num_training_steps)
print(num_training_steps)

1377


E:\Anaconda\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


编写pytorch training loops，思路如下：

* .for每一个epoch
* *.从dataloader里取出一个个batch
* 3.把batch喂给model（先把batch都移动到对应的device上）
* 4.拿出loss，进行反向传播backward
* 5.分别把optimizer和scheduler都更新一个stepl一个step
最后别忘了每次更新都要清空grad，即对optimizer进行zero_grad()操作。

In [ ]:
--------------------------------------------------------------

In [ ]:
# 7、Pytorch training loops
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

from tqdm import tqdm

for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader):
        # 要在GPU上训练，需要把数据集都移动到GPU上：
        batch = {k:v.to(device) for k,v in batch.items()}
        loss = model(**batch).loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

In [ ]:
# 8、Evaluation
from datasets import load_metric

metric= load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():  # evaluation的时候不需要算梯度
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    # 由于dataloader是每次输出一个batch，因此我们要等着把所有batch都添加进来，再进行计算
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()